# Get Constelation Data

In [ ]:
import os
import requests
import zipfile
import pandas as pd

def get_file(url, path):
    if os.path.exists(path):
        print("Skipping download {} already exists.".format(path))
        return None
    print("Downloading {} to {} ...".format(url, path))
    directory, file = os.path.split(os.path.abspath(path))
    if not os.path.exists(directory):
        os.mkdir(directory)
    r = requests.get(url, allow_redirects=True, verify=False)
    with open(path, "wb") as f:
        f.write(r.content)
    print("Complete")
        
def unzip(path, dest):
    if os.path.exists(dest):
        print("Skipping unzip as {} already exists.".format(dest))
        return None
    os.mkdir(dest)
    with zipfile.ZipFile(path, "r") as zip_ref:
        zip_ref.extractall(dest)

In [ ]:
# populate this with url to your data
data_url = "****"
data_path = os.path.join(os.getcwd(), "datasets/constelation.csv")

In [ ]:
get_file(data_url, data_path)

# Wrangle Dataset

This will be quite specific to our data so you may need to modify this to your liking.

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv(data_path).drop("totyield", axis=1).dropna().sort_values(by=["milk_date"])
data

In [ ]:
i = 0
signals = []
window_size = 20
for group in data.groupby(["itb"]):
#     print(group)
    example = group[1].select_dtypes(include=[np.number])
    for window in example.rolling(window=window_size):

        if len(window["milkyield"]) == window_size:
            signals.append(window)
        i += 1
#     for window in example.rolling(window=20):
        
# #     for window in example.rolling(window=20):
#         if i <=30 and i >=20:
#             print(window)
#         i += 1
#         pass
print(len(signals))
# print(window)
# print(type(window))

#     x = group.select_dtypes(include=[np.number])
#     print(x)
#     for window in x.rolling(window=20):
#         print(window)

# Get Constelation Model

In [ ]:
from fhez.nn.graph.prefab import cnn_regressor

network = cnn_regressor(data_shape=signals[0].shape, filter_length=5, stride=3)

In [ ]:
import copy
def strip(graph):
    g = copy.deepcopy(graph)
    for node in g.nodes(data=True):
        try:
            # node[1]["title"] = "{}:\n{}".format(type(node[1]["node"]), repr(node[1]["node"]))
            del node[1]["node"]
        except KeyError:
            pass
    return g

In [ ]:
from pyvis.network import Network
stripped = strip(network)
print(stripped)

from pyvis.network import Network
net = Network('700px', '700px', bgcolor='#222222', font_color='white', notebook=True)
net.from_nx(stripped)
# net.show_buttons(filter_="physics")
net.show("constelation.html")

# Train Model